In [27]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('/home/ec2-user/SageMaker', 'factcheck-model-v2'))

if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-12-09 04:25:22.391596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 04:25:22.599884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 04:25:22.664104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 04:25:23.091463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 04:25:25.701668: W tensorflow/comp

In [2]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [ ]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']

    train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    if use_only_card:
        print("using card")
        train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)

    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': size_of_dataset,
        'use_only_CARD': use_only_card
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)

/home/ec2-user/SageMaker/factcheck-model-v2/Testing/DataLoader.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.GoogleFactCheckData['Numerical Rating'] = self.GoogleFactCheckData['Numerical Rating'].replace({True: 3, False: 1})


All data sources are being used
1
Number of experiments to run:  1


/home/ec2-user/SageMaker/factcheck-model-v2/Testing/DataLoader.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.GoogleFactCheckData['Numerical Rating'] = self.GoogleFactCheckData['Numerical Rating'].replace({True: 3, False: 1})


Numerical Rating
3    3256
1      28
Name: count, dtype: int64
Numerical Rating
3    771
1     51
Name: count, dtype: int64
using card
All data sources are being used
1
Numerical Rating
3    4414
1    2516
Name: count, dtype: int64
Numerical Rating
3    894
1    839
Name: count, dtype: int64
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  1739
Getting embeddings for batch  75  out of  1739
Getting embeddings for batch  150  out of  1739
Getting embeddings for batch  225  out of  1739
Getting embeddings for batch  300  out of  1739
Getting embeddings for batch  375  out of  1739
Getting embeddings for batch  450  out of  1739
Getting embeddings for batch  525  out of  1739
Getting embeddings for batch  600  out of  1739
Getting embeddings for batch  675  out of  1739
Getting embeddings for batch  750  out of  1739
Getting embeddings for batch  825  out of  1739
Getting embeddings for batch  900  out of  1739
Getting embeddings for batch  975  out of  1739
Getting 

2024-12-09 04:27:27.151366: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-09 04:27:27.155825: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 178790400 exceeds 10% of free system memory.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [30]:
results_df = pd.json_normalize(results)
# Get current date and time
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
results_df
# results_df.to_csv(f'results_${str(now)}.csv', index=False)

""


In [7]:
best_estimator.clusters_df
# Filter clusters_df to where cluster is not -1 and predicted_cluster is 4
best_estimator.clusters_df[(best_estimator.clusters_df['cluster'] != -1) & (best_estimator.clusters_df['predicted_veracity'] == 4)]
# Filter to cluster 144
best_estimator.clusters_df[best_estimator.clusters_df['cluster'] == 144]

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy
74,"Compared to 2022, the 2023 data show a 25% dec...",1,True,4,"[2.463346242904663, 2.7957773208618164, 9.9586...",144,0,5,0.0
378,Improvements in existing plants reduced steel-...,3,True,4,"[2.42880916595459, 2.778208017349243, 9.946139...",144,0,5,0.0
438,Reported power plant emissions increased drama...,1,True,4,"[2.5143368244171143, 2.7544777393341064, 9.951...",144,0,5,0.0
649,"In 2022, reported emissions from large industr...",1,True,4,"[2.4696571826934814, 2.7592506408691406, 9.971...",144,0,5,0.0
862,"Between 1990 and 2023, annual emissions of SO2...",1,True,4,"[2.4708685874938965, 2.7929885387420654, 9.978...",144,0,5,0.0


In [ ]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
df_with_two_dimens

# S3 downloader

In [9]:
# file in and out

import boto3

# create s3 client
s3 = boto3.client('s3')

bucket_name = 'sagemaker-us-east-1-390403859474'

s3_file_key = 'processed_file.csv'

local_file_path = 'downloaded_file.csv'

# download the file
s3.download_file(bucket_name, s3_file_key, local_file_path)

print(f"File {s3_file_key} has been downloaded to {local_file_path}.")

processed_file_path = f'results_${str(now)}.csv'
results_df.to_csv(processed_file_path, index=False)

# define upload path
upload_file_key = 'processed_file.csv'

# upload the file
s3.upload_file(processed_file_path, bucket_name, upload_file_key)

print(f"File {processed_file_path} has been uploaded to s3://{bucket_name}/{upload_file_key}.")

File processed_file.csv has been downloaded to downloaded_file.csv.
File results_$2024-11-22_04-36-56.csv has been uploaded to s3://sagemaker-us-east-1-390403859474/processed_file.csv.


# Zapier

In [8]:
# best_estimator.clusters_df.to_csv("test1.csv", index=False)

In [ ]:
import pandas as pd
import os
import boto3
from datetime import datetime
import re
import time

# AWS S3 Setup
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
BUCKET_NAME = 'testtestgfc'

# Data File
CSV_FILE_PATH = 'test1.csv'

# Connect to S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY
)

# Create local temp file
TEMP_DIR = 'temp_files'
os.makedirs(TEMP_DIR, exist_ok=True)

# Read CSV file and get column name
data = pd.read_csv(CSV_FILE_PATH)# .iloc[41:50]

batch_size = 10
total_batches = (len(data) + batch_size - 1) // batch_size


for i in range(total_batches):
    
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(data))
    df = data.iloc[start_idx:end_idx]
    column_names = df.columns
    
    # Create files contains single claim
    for index, row in df.iterrows():
        text = str(df.loc[index, 'text'])
        filtered_text = ''.join(re.findall(r'[A-Za-z0-9]', text))
        sub_name = filtered_text[:30]
        file_name = f"claim_{sub_name}.csv"
        file_path = os.path.join(TEMP_DIR, file_name)
        
        single_row_df = pd.DataFrame([row], columns=column_names)
        single_row_df.to_csv(file_path, index=False)
        
        # Upload to S3
        s3_client.upload_file(
            Filename=file_path,
            Bucket=BUCKET_NAME,
            Key=f"groundtruth/{file_name}"  # S3 path
        )
        print(f"Uploaded {file_name} to S3.")

    print("Rest for 180s...")
    time.sleep(180)

# Delete temp files
for file in os.listdir(TEMP_DIR):
    os.remove(os.path.join(TEMP_DIR, file))
os.rmdir(TEMP_DIR)

print("All files uploaded successfully!")

Uploaded claim_InhisRMSarticleKevinTrenbertha.csv to S3.
Uploaded claim_Fromthebeginningstagesofthepre.csv to S3.
Uploaded claim_Withover14millionAmericansunem.csv to S3.
Uploaded claim_Theglobaltemperaturehasbeenris.csv to S3.
Uploaded claim_Theaverageperyearbetween1971an.csv to S3.
Uploaded claim_Globalairmilesexplodedby1960an.csv to S3.
Uploaded claim_Howeverbecausetheyweredesigned.csv to S3.
Uploaded claim_Putanotherwayoverthepastdecade.csv to S3.
Uploaded claim_Thesetwoscientistsreviewedmore.csv to S3.
Uploaded claim_Climateactivistsvowedtokillmil.csv to S3.
Rest for 180s...
Uploaded claim_MrRameshsaidtherateofretreatof.csv to S3.
Uploaded claim_TheNSIDCmapsshow7moreicein2010.csv to S3.
Uploaded claim_Almost1millionvoterscastballot.csv to S3.
Uploaded claim_Thelatterwasatwominutesurveyse.csv to S3.
Uploaded claim_TheUnitedStateshasenoughcoalre.csv to S3.
Uploaded claim_Analysisof20yearsofsatelliteda.csv to S3.
Uploaded claim_YesterdaytheClimateInstitutere.csv to S3.
Uploaded claim

KeyboardInterrupt: 

# DataReader

In [ ]:
s3://sagemaker-us-east-1-390403859474/AWSDynamoDB/01732296933050-72938614/data/ovqkiyhsh42fzdpaglcq36re7m.json.gz

In [17]:
import boto3
import pandas as pd
import gzip
import json

# define S3 bucket and object key
bucket = "sagemaker-us-east-1-390403859474"
key = "AWSDynamoDB/01732296933050-72938614/data/ovqkiyhsh42fzdpaglcq36re7m.json.gz"

# use the boto3 client
s3 = boto3.client('s3')

# download the file locally
s3.download_file(bucket, key, 'data.json.gz')

# unzip and read line by line
data = []
with gzip.open('data.json.gz', 'rt') as f:
    for line in f:
        data.append(json.loads(line))

# transform to DataFrame
df = pd.json_normalize(data)

df.to_csv("GoogleFactCheckData.csv")

# show the data
print(df.head())


                                     Item.claim_id.S Item.language_code.S  \
0  https://factcheck.afp.com/bidens-climate-plan-...                   en   
1  https://www.usatoday.com/story/news/factcheck/...                   en   
2  https://apnews.com/article/fact-check-hillary-...                   en   
3  https://www.aap.com.au/factcheck/senator-blowi...                   en   
4  https://www.usatoday.com/story/news/factcheck/...                   en   

                                 Item.review_title.S          Item.claimant.S  \
0  Biden’s climate plan does not target US meat c...                 Fox News   
1  Is global warming caused by humans? Evidence s...             Social media   
2  No, Hillary Clinton didn’t call for banning ca...       social media users   
3  Senator blowing hot air with climate science d...  Senator Malcolm Roberts   
4  Humans have significant impact on atmospheric ...             Social media   

      Item.claim_date.S                           